In [2]:
import pandas as pd 
import numpy as np 
import os
import xml.etree.ElementTree as ET

In [39]:
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    image_path = root.find("path").text if root.find("path") is not None else ""
    time_labeled = root.find("time_labeled").text if root.find("time_labeled") is not None else ""
    labeled = root.find("labeled").text if root.find("labeled") is not None else ""
    
    size = root.find("size")
    width = size.find("width").text if size is not None else ""
    height = size.find("height").text if size is not None else ""
    depth = size.find("depth").text if size is not None else ""
    
    objects = root.findall("outputs/object/item")
    
    data = []
    for obj in objects:
        name = obj.find("name").text if obj.find("name") is not None else ""
        bndbox = obj.find("bndbox")
        polygon = obj.find("polygon")
        bezier = obj.find("cubic_bezier")

        bndbox_str, polygon_str, bezier_str = np.nan, np.nan, np.nan

        # Handle Bounding Box (bndbox)
        if bndbox is not None:
            xmin = bndbox.find("xmin").text if bndbox.find("xmin") is not None else np.nan
            ymin = bndbox.find("ymin").text if bndbox.find("ymin") is not None else np.nan
            xmax = bndbox.find("xmax").text if bndbox.find("xmax") is not None else np.nan
            ymax = bndbox.find("ymax").text if bndbox.find("ymax") is not None else np.nan
            bndbox_str = f"({xmin}, {ymin}, {xmax}, {ymax})"

        # Handle Polygon (polygon)
        if polygon is not None:
            polygon_points = []
            for i in range(1, 50):  # Assuming max 50 points in a polygon
                x_tag = f"x{i}"
                y_tag = f"y{i}"
                x_val = polygon.find(x_tag)
                y_val = polygon.find(y_tag)
                
                if x_val is not None and y_val is not None:
                    polygon_points.append(f"({x_val.text}, {y_val.text})")
                else:
                    break  # Stop when no more points

            polygon_str = "; ".join(polygon_points) if polygon_points else np.nan

        # Handle Bézier Curve (cubic_bezier)
        if bezier is not None:
            bezier_points = []
            for i in range(1, 50):  # Assuming max 50 control points in a curve
                x_tag = f"x{i}"
                y_tag = f"y{i}"
                x_c1_tag = f"x{i}_c1"
                y_c1_tag = f"y{i}_c1"
                x_c2_tag = f"x{i}_c2"
                y_c2_tag = f"y{i}_c2"

                x_val = bezier.find(x_tag)
                y_val = bezier.find(y_tag)
                x_c1_val = bezier.find(x_c1_tag)
                y_c1_val = bezier.find(y_c1_tag)
                x_c2_val = bezier.find(x_c2_tag)
                y_c2_val = bezier.find(y_c2_tag)

                if x_val is not None and y_val is not None and x_c1_val is not None and y_c1_val is not None and x_c2_val is not None and y_c2_val is not None:
                    bezier_points.append(f"({x_val.text}, {y_val.text}, {x_c1_val.text}, {y_c1_val.text}, {x_c2_val.text}, {y_c2_val.text})")
                else:
                    break  # Stop when no more points

            bezier_str = "; ".join(bezier_points) if bezier_points else np.nan

        # Append data ensuring all columns are present
        data.append([
            os.path.basename(file_path), image_path, name, 
            width, height, depth, labeled, time_labeled, 
            bndbox_str, polygon_str, bezier_str
        ])

    return data

def process_folder(folder_path, output_csv):
    all_data = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".xml"):
            file_path = os.path.join(folder_path, file_name)
            all_data.extend(parse_xml(file_path))
    
    columns = ["file_name", "image_path", "label_name", "width", "height", "depth", "label", "timestamp", "bndbox", "polygon", "bezier"]
    df = pd.DataFrame(all_data, columns=columns)
    df.to_csv(output_csv, index=False, encoding='utf-8')
    print(f"CSV file saved: {output_csv}")

# Example usage
folder_path = "data/annotations"  # Change this to your actual folder path
output_csv = "output.csv"
process_folder(folder_path, output_csv)

CSV file saved: output.csv


In [40]:
df = pd.read_csv('output.csv')
print(df['image_path'].nunique())
df.tail(5)

8037


,file_name,image_path,label_name,width,height,depth,label,timestamp,bndbox,polygon,bezier
15788,2120340_0307.xml,2120340\0307.png,异常,2048,2048,3,True,1592461776764,NaN,"(244, 1026); (271, 1037); (285, 1051); (303, 1...",NaN
15789,2120340_0313.xml,2120340\0313.png,异常,2048,2048,3,True,1592461836496,"(1327, 1211, 1577, 1305)",NaN,NaN
15790,2120340_0313.xml,2120340\0313.png,异常,2048,2048,3,True,1592461836496,NaN,"(1564, 1323); (1569, 1344); (1561, 1361); (154...",NaN
15791,2120340_0313.xml,2120340\0313.png,异常,2048,2048,3,True,1592461836496,NaN,"(1381, 1393); (1364, 1383); (1337, 1371); (132...",NaN
15792,2120340_0322.xml,2120340\0322.png,异常,2048,2048,3,True,1592461859743,"(1896, 1680, 2022, 1920)",NaN,NaN


In [41]:
test = df[df['file_name']=='2120340_0299.xml']
test

,file_name,image_path,label_name,width,height,depth,label,timestamp,bndbox,polygon,bezier
15782,2120340_0299.xml,2120340\0299.png,异常,2048,2048,3,True,1592461687766,NaN,NaN,"(1274, 680, 1274, 680, 1274, 679); (1288, 658,..."
15783,2120340_0299.xml,2120340\0299.png,异常,2048,2048,3,True,1592461687766,NaN,NaN,"(1215, 862, 1215, 862, 1214, 861); (1203, 823,..."
15784,2120340_0299.xml,2120340\0299.png,异常,2048,2048,3,True,1592461687766,NaN,NaN,"(1141, 990, 1141, 990, 1140, 989); (1166, 1019..."
